<a href="https://colab.research.google.com/github/Mwauracollins/holy-gpt/blob/main/src/bigram/nnModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from tiktoken import get_encoding
tokenizer = get_encoding("cl100k_base")

In [3]:
input_file = "drive/MyDrive/Datasets/recompiled.txt"

In [4]:
words = open(input_file, "r").read().splitlines()

In [5]:
tokens = [tokenizer.encode(word) for word in words]

In [6]:
vocab_size = tokenizer.n_vocab

In [7]:
block_size = 3
X, Y = [], []

for token_list in tokens:
    context = [0] * block_size
    for token in token_list + [tokenizer.eot_token]:
        X.append(context)
        Y.append(token)
        context = context[1:] + [token]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [8]:
X.shape, Y.shape, X.dtype, Y.dtype

(torch.Size([1058610, 3]), torch.Size([1058610]), torch.int64, torch.int64)

In [9]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for token_list in tokens:
        context = [0] * block_size
        for token in token_list + [tokenizer.eot_token]:
            X.append(context)
            Y.append(token)
            context = context[1:] + [token]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(tokens)
n1 = int(0.8*len(tokens))
n2 = int(0.9*len(tokens))

Xtr, Ytr = build_dataset(tokens[:n1])
Xdev, Ydev = build_dataset(tokens[n1:n2])
Xte, Yte = build_dataset(tokens[n2:])
print(Xtr.shape, Ytr.shape, Xdev.shape, Ydev.shape, Xte.shape, Yte.shape)

torch.Size([1058610, 3]) torch.Size([1058610])
torch.Size([1058610, 3]) torch.Size([1058610])
torch.Size([1058610, 3]) torch.Size([1058610])
torch.Size([1058610, 3]) torch.Size([1058610]) torch.Size([1058610, 3]) torch.Size([1058610]) torch.Size([1058610, 3]) torch.Size([1058610])


In [10]:
embedding_dim = 10

In [11]:
g = torch.Generator().manual_seed(21)
C = torch.randn((vocab_size, embedding_dim), generator=g)
W1 = torch.randn((block_size * embedding_dim, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, vocab_size), generator=g)
b2 = torch.randn(vocab_size, generator=g)
parameters = [C, W1, b1, W2, b2]

In [12]:
sum(p.nelement() for p in parameters)


21164647

In [13]:
for p in parameters:
    p.requires_grad = True

In [14]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [15]:
lri = []
lossi = []
stepi = []

In [ ]:
for i in range(200000):
    ix = torch.randint(0, Xtr.shape[0], (32,))

    #forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, block_size * embedding_dim) @ W1 + b1)
    logits = h @ W2 + b2
    loss = torch.nn.functional.cross_entropy(logits, Ytr[ix])

    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update

    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    stepi.append(i)
    # lri.append(lr)
    lossi.append(loss.log10().item())

In [ ]:
plt.plot(stepi, lossi)

In [ ]:
# emb = C[Xtr]
# h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
# logits = h @ W2 + b2
# loss = torch.nn.functional.cross_entropy(logits, Ytr)
# loss

In [ ]:
# emb = C[Xdev]
# h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
# logits = h @ W2 + b2
# loss = torch.nn.functional.cross_entropy(logits, Ydev)
# loss

In [ ]:
# context = [0] * block_size
# C[torch.tensor(context)].shape

In [ ]:
import torch.nn.functional as F
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,embedding_dim)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(tokenizer.decode(out))

In [ ]:
int_to_str = {i: tokenizer.decode([i]) for i in range(vocab_size)}

In [ ]:
plt.figure(figsize=(12, 12))
plt.scatter(C[:200, 0].data, C[:200, 1].data, s=200)

for i in range(200):
    plt.text(C[i, 0].item(), C[i, 1].item(), int_to_str[i], ha="center", va="center", color='black')

plt.grid('minor')
plt.show()